In [1]:
import xarray as xr
import pint_xarray
import pandas as pd
from openscm_units import unit_registry as ureg
pint_xarray.accessors.setup_registry(ureg)

## read the data, convert to xarray, and add metadata

This is the same for the two proposals, you can skip to the next headline

In [2]:
raw = (
    pd
    .read_csv('fao_emi_total.csv')
    .to_xarray()
    .set_index({'index': ['Area', 'Date', 'Element', 'Item']})
    .unstack()
)

In [3]:
ds = raw['Emissions/gigagram'].to_dataset('Element')

ds['CH4'].attrs = {
    'entity': 'CH4',
    'units': 'Gg CH4 / year',
}
ds['CH4 (CO2eq)'].attrs = {
    'entity': 'CH4',
    'units': 'Gg CO2 / year',
    'gwp conversions': 'SARGWP100',  # I don't know if SARGWP100 is correct here, this is just an example
}
ds['CO2eq'].attrs = {
    'entity': 'KYOTOGHG',
    'units': 'Gg CO2 / year',
    'gwp conversions': 'SARGWP100',  # I don't know if SARGWP100 is correct here, this is just an example
}                      
ds['N2O'].attrs = {
    'entity': 'N2O',
    'units': 'Gg N2O / year',
}
ds['N2O (CO2eq)'].attrs = {
    'entity': 'N2O',
    'units': 'Gg CO2 / year',
    'gwp conversions': 'SARGWP100',  # I don't know if SARGWP100 is correct here, this is just an example
}

In [4]:
ds = ds.pint.quantify(unit_registry=ureg)
ds

Magnitude,[[[nan 285.7793 4.0734 ... nan 29.4 nan] [nan 290.8671 4.1933 ... nan 29.4 nan] [nan 302.0282 4.1933 ... nan 29.4 nan] ... [nan 426.2681 3.0734 ... nan 16.66 nan] [nan 517.4886 3.4935 ... nan 32.3685 nan] [nan 684.3751 3.4857 ... nan 41.3331 nan]] [[nan 6488.4324 54.5926 ... nan 342.8331 nan] [nan 6614.9865 56.1299 ... nan 397.6844 nan] [nan 6719.5024 56.994 ... nan 399.0665 nan] ... [nan 23774.0131 141.2433 ... nan 1682.9586 nan] [nan 21812.255 131.0784 ... nan 1468.39 nan] [nan 25326.3427 150.9778 ... nan 1789.5675 nan]] [[nan 59.0109 0.5771 ... nan 0.8308 nan] [nan 59.5791 0.5209 ... nan 0.9542 nan] [nan 59.3533 0.5344 ... nan 1.0839 nan] ... [nan 83.8931 0.2342 ... nan 0.0 nan] [nan 101.2206 0.2062 ... nan nan nan] [nan 100.5291 0.2081 ... nan nan nan]] ... [[nan 687.7415 8.9146 ... nan 1.5736 nan] [nan 696.0861 8.9658 ... nan 1.5624 nan] [nan 645.8357 8.8342 ... nan 1.7024 nan] ... [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan]] [[nan 46.2113 2.0254 ... nan nan nan] [nan 45.9146 2.0256 ... nan nan nan] [nan 45.9272 2.0794 ... nan nan nan] ... [nan 501.15 3.2584 ... nan 1.5461 nan] [nan 462.0483 2.1085 ... nan 1.5187 nan] [nan 498.4685 2.5119 ... nan 1.9286 nan]] [[nan 130.7866 2.3037 ... nan 0.07 nan] [nan 133.1604 2.1732 ... nan 0.07 nan] [nan 136.8973 2.0478 ... nan 0.14 nan] ... [nan 264.4934 3.8013 ... nan 0.0225 nan] [nan 378.0923 5.3065 ... nan 0.03 nan] [nan 482.2404 4.8239 ... nan 0.0355 nan]]]
Units,CH4 gigagram/year
Magnitude,[[[nan 6001.3654 85.5416 ... nan 617.4 nan] [nan 6108.2101 88.0591 ... nan 617.4 nan] [nan 6342.5929 88.0591 ... nan 617.4 nan] ... [nan 8951.6301 64.5416 ... nan 349.86 nan] [nan 10867.26 73.363 ... nan 679.7382 nan] [nan 14371.8766 73.1997 ... nan 867.9956 nan]] [[nan 136257.0805 1146.4437 ... nan 7199.4953 nan] [nan 138914.7165 1178.7272 ... nan 8351.3734 nan] [nan 141109.5508 1196.8745 ... nan 8380.3971 nan] ... [nan 499254.2755 2966.1094 ... nan 35342.1316 nan] [nan 458057.3555 2752.6465 ... nan 30836.1898 nan] [nan 531853.1974 3170.5338 ... nan 37580.9168 nan]] [[nan 1239.2292 12.1195 ... nan 17.445999999999998 nan] [nan 1251.1616 10.9383 ... nan 20.039 nan] [nan 1246.4197 11.2227 ... nan 22.7615 nan] ... [nan 1761.7543 4.9177 ... nan 0.0 nan] [nan 2125.632 4.3295 ... nan nan nan] [nan 2111.111 4.3692 ... nan nan nan]] ... [[nan 14442.5724 187.2059 ... nan 33.0456 nan] [nan 14617.8086 188.2819 ... nan 32.8104 nan] [nan 13562.5496 185.5192 ... nan 35.7504 nan] ... [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan]] [[nan 970.4377 42.5336 ... nan nan nan] [nan 964.2064 42.5376 ... nan nan nan] [nan 964.4717 43.6671 ... nan nan nan] ... [nan 10524.1495 68.426 ... nan 32.4679 nan] [nan 9703.0137 44.2775 ... nan 31.8934 nan] [nan 10467.838 52.7504 ... nan 40.5009 nan]] [[nan 2746.5195 48.3787 ... nan 1.47 nan] [nan 2796.3674 45.6378 ... nan 1.47 nan] [nan 2874.8426 43.0037 ... nan 2.94 nan] ... [nan 5554.3619 79.8269 ... nan 0.4715 nan] [nan 7939.9378 111.4355 ... nan 0.6293 nan] [nan 10127.0484 101.3024 ... nan 0.7457 nan]]]
Units,CO2 gigagram/year
Magnitude,[[[3097.8213 9255.678 118.2797 ... 2368.1734 617.4 6.4546] [3063.8992 9331.0188 121.7607 ... 2326.1498 617.4 6.4546] [3116.2194 9620.3601 121.7607 ... 2390.2709 617.4 6.4546] ... [4732.3633 13832.9964 89.2428 ... 3159.8598 349.86 623.9758] [5717.29 16810.3052 101.4402 ... 4200.1834 679.7382 275.6073] [7962.9146 22644.1569 101.2144 ... 5780.8189 867.9956 511.3241]] [[95074.6375 233438.4999 1585.2062 ... 86108.4167 7199.4953 2277.8822] [96598.8864 237662.8733 1629.845 ... 86991.4047 8351.3734 2454.3328] [98584.3345 241883.7194 1654.9376 ... 88348.6382 8380.3971 2868.7402] ... [324796.6377 936234.3419 4101.287 ... 263095.5966 35342.1316 25091.0376] [319774.7461 881918.151 3806.1285 ... 250118.2627 30836.1898 31441.5307] [398749.5427 1039555.2658 4383.948 ... 303100.5699 37580.9168 46671.6549]] [[565.3851 1903.8378 16.7578 ... 378.0223 17.445999999

## name the coordinates properly and add meta-information about used terminologies

Here, the two proposals differ.

**In proposal 1, we will rename 'Item' so that it includes the information
about the used termonology, and will add meta-information that this dimension contains the information
about the sector.**

In [5]:
proposal1 = ds.rename({'Item': 'Sector (FAO terminology)'})
proposal1.attrs['sector_dimension'] = 'Sector (FAO terminology)'
proposal1

Magnitude,[[[nan 285.7793 4.0734 ... nan 29.4 nan] [nan 290.8671 4.1933 ... nan 29.4 nan] [nan 302.0282 4.1933 ... nan 29.4 nan] ... [nan 426.2681 3.0734 ... nan 16.66 nan] [nan 517.4886 3.4935 ... nan 32.3685 nan] [nan 684.3751 3.4857 ... nan 41.3331 nan]] [[nan 6488.4324 54.5926 ... nan 342.8331 nan] [nan 6614.9865 56.1299 ... nan 397.6844 nan] [nan 6719.5024 56.994 ... nan 399.0665 nan] ... [nan 23774.0131 141.2433 ... nan 1682.9586 nan] [nan 21812.255 131.0784 ... nan 1468.39 nan] [nan 25326.3427 150.9778 ... nan 1789.5675 nan]] [[nan 59.0109 0.5771 ... nan 0.8308 nan] [nan 59.5791 0.5209 ... nan 0.9542 nan] [nan 59.3533 0.5344 ... nan 1.0839 nan] ... [nan 83.8931 0.2342 ... nan 0.0 nan] [nan 101.2206 0.2062 ... nan nan nan] [nan 100.5291 0.2081 ... nan nan nan]] ... [[nan 687.7415 8.9146 ... nan 1.5736 nan] [nan 696.0861 8.9658 ... nan 1.5624 nan] [nan 645.8357 8.8342 ... nan 1.7024 nan] ... [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan]] [[nan 46.2113 2.0254 ... nan nan nan] [nan 45.9146 2.0256 ... nan nan nan] [nan 45.9272 2.0794 ... nan nan nan] ... [nan 501.15 3.2584 ... nan 1.5461 nan] [nan 462.0483 2.1085 ... nan 1.5187 nan] [nan 498.4685 2.5119 ... nan 1.9286 nan]] [[nan 130.7866 2.3037 ... nan 0.07 nan] [nan 133.1604 2.1732 ... nan 0.07 nan] [nan 136.8973 2.0478 ... nan 0.14 nan] ... [nan 264.4934 3.8013 ... nan 0.0225 nan] [nan 378.0923 5.3065 ... nan 0.03 nan] [nan 482.2404 4.8239 ... nan 0.0355 nan]]]
Units,CH4 gigagram/year
Magnitude,[[[nan 6001.3654 85.5416 ... nan 617.4 nan] [nan 6108.2101 88.0591 ... nan 617.4 nan] [nan 6342.5929 88.0591 ... nan 617.4 nan] ... [nan 8951.6301 64.5416 ... nan 349.86 nan] [nan 10867.26 73.363 ... nan 679.7382 nan] [nan 14371.8766 73.1997 ... nan 867.9956 nan]] [[nan 136257.0805 1146.4437 ... nan 7199.4953 nan] [nan 138914.7165 1178.7272 ... nan 8351.3734 nan] [nan 141109.5508 1196.8745 ... nan 8380.3971 nan] ... [nan 499254.2755 2966.1094 ... nan 35342.1316 nan] [nan 458057.3555 2752.6465 ... nan 30836.1898 nan] [nan 531853.1974 3170.5338 ... nan 37580.9168 nan]] [[nan 1239.2292 12.1195 ... nan 17.445999999999998 nan] [nan 1251.1616 10.9383 ... nan 20.039 nan] [nan 1246.4197 11.2227 ... nan 22.7615 nan] ... [nan 1761.7543 4.9177 ... nan 0.0 nan] [nan 2125.632 4.3295 ... nan nan nan] [nan 2111.111 4.3692 ... nan nan nan]] ... [[nan 14442.5724 187.2059 ... nan 33.0456 nan] [nan 14617.8086 188.2819 ... nan 32.8104 nan] [nan 13562.5496 185.5192 ... nan 35.7504 nan] ... [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan]] [[nan 970.4377 42.5336 ... nan nan nan] [nan 964.2064 42.5376 ... nan nan nan] [nan 964.4717 43.6671 ... nan nan nan] ... [nan 10524.1495 68.426 ... nan 32.4679 nan] [nan 9703.0137 44.2775 ... nan 31.8934 nan] [nan 10467.838 52.7504 ... nan 40.5009 nan]] [[nan 2746.5195 48.3787 ... nan 1.47 nan] [nan 2796.3674 45.6378 ... nan 1.47 nan] [nan 2874.8426 43.0037 ... nan 2.94 nan] ... [nan 5554.3619 79.8269 ... nan 0.4715 nan] [nan 7939.9378 111.4355 ... nan 0.6293 nan] [nan 10127.0484 101.3024 ... nan 0.7457 nan]]]
Units,CO2 gigagram/year
Magnitude,[[[3097.8213 9255.678 118.2797 ... 2368.1734 617.4 6.4546] [3063.8992 9331.0188 121.7607 ... 2326.1498 617.4 6.4546] [3116.2194 9620.3601 121.7607 ... 2390.2709 617.4 6.4546] ... [4732.3633 13832.9964 89.2428 ... 3159.8598 349.86 623.9758] [5717.29 16810.3052 101.4402 ... 4200.1834 679.7382 275.6073] [7962.9146 22644.1569 101.2144 ... 5780.8189 867.9956 511.3241]] [[95074.6375 233438.4999 1585.2062 ... 86108.4167 7199.4953 2277.8822] [96598.8864 237662.8733 1629.845 ... 86991.4047 8351.3734 2454.3328] [98584.3345 241883.7194 1654.9376 ... 88348.6382 8380.3971 2868.7402] ... [324796.6377 936234.3419 4101.287 ... 263095.5966 35342.1316 25091.0376] [319774.7461 881918.151 3806.1285 ... 250118.2627 30836.1898 31441.5307] [398749.5427 1039555.2658 4383.948 ... 303100.5699 37580.9168 46671.6549]] [[565.3851 1903.8378 16.7578 ... 378.0223 17.445999999

**In proposal 2, we will rename 'Item' to the canonical name `sector`, and will add meta-information about
the used terminology.**

In [6]:
proposal2 = ds.rename({'Item': 'sector'})
proposal2.attrs['sector_terminology'] = 'FAO terminology'
proposal2

Magnitude,[[[nan 285.7793 4.0734 ... nan 29.4 nan] [nan 290.8671 4.1933 ... nan 29.4 nan] [nan 302.0282 4.1933 ... nan 29.4 nan] ... [nan 426.2681 3.0734 ... nan 16.66 nan] [nan 517.4886 3.4935 ... nan 32.3685 nan] [nan 684.3751 3.4857 ... nan 41.3331 nan]] [[nan 6488.4324 54.5926 ... nan 342.8331 nan] [nan 6614.9865 56.1299 ... nan 397.6844 nan] [nan 6719.5024 56.994 ... nan 399.0665 nan] ... [nan 23774.0131 141.2433 ... nan 1682.9586 nan] [nan 21812.255 131.0784 ... nan 1468.39 nan] [nan 25326.3427 150.9778 ... nan 1789.5675 nan]] [[nan 59.0109 0.5771 ... nan 0.8308 nan] [nan 59.5791 0.5209 ... nan 0.9542 nan] [nan 59.3533 0.5344 ... nan 1.0839 nan] ... [nan 83.8931 0.2342 ... nan 0.0 nan] [nan 101.2206 0.2062 ... nan nan nan] [nan 100.5291 0.2081 ... nan nan nan]] ... [[nan 687.7415 8.9146 ... nan 1.5736 nan] [nan 696.0861 8.9658 ... nan 1.5624 nan] [nan 645.8357 8.8342 ... nan 1.7024 nan] ... [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan]] [[nan 46.2113 2.0254 ... nan nan nan] [nan 45.9146 2.0256 ... nan nan nan] [nan 45.9272 2.0794 ... nan nan nan] ... [nan 501.15 3.2584 ... nan 1.5461 nan] [nan 462.0483 2.1085 ... nan 1.5187 nan] [nan 498.4685 2.5119 ... nan 1.9286 nan]] [[nan 130.7866 2.3037 ... nan 0.07 nan] [nan 133.1604 2.1732 ... nan 0.07 nan] [nan 136.8973 2.0478 ... nan 0.14 nan] ... [nan 264.4934 3.8013 ... nan 0.0225 nan] [nan 378.0923 5.3065 ... nan 0.03 nan] [nan 482.2404 4.8239 ... nan 0.0355 nan]]]
Units,CH4 gigagram/year
Magnitude,[[[nan 6001.3654 85.5416 ... nan 617.4 nan] [nan 6108.2101 88.0591 ... nan 617.4 nan] [nan 6342.5929 88.0591 ... nan 617.4 nan] ... [nan 8951.6301 64.5416 ... nan 349.86 nan] [nan 10867.26 73.363 ... nan 679.7382 nan] [nan 14371.8766 73.1997 ... nan 867.9956 nan]] [[nan 136257.0805 1146.4437 ... nan 7199.4953 nan] [nan 138914.7165 1178.7272 ... nan 8351.3734 nan] [nan 141109.5508 1196.8745 ... nan 8380.3971 nan] ... [nan 499254.2755 2966.1094 ... nan 35342.1316 nan] [nan 458057.3555 2752.6465 ... nan 30836.1898 nan] [nan 531853.1974 3170.5338 ... nan 37580.9168 nan]] [[nan 1239.2292 12.1195 ... nan 17.445999999999998 nan] [nan 1251.1616 10.9383 ... nan 20.039 nan] [nan 1246.4197 11.2227 ... nan 22.7615 nan] ... [nan 1761.7543 4.9177 ... nan 0.0 nan] [nan 2125.632 4.3295 ... nan nan nan] [nan 2111.111 4.3692 ... nan nan nan]] ... [[nan 14442.5724 187.2059 ... nan 33.0456 nan] [nan 14617.8086 188.2819 ... nan 32.8104 nan] [nan 13562.5496 185.5192 ... nan 35.7504 nan] ... [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan] [nan nan nan ... nan nan nan]] [[nan 970.4377 42.5336 ... nan nan nan] [nan 964.2064 42.5376 ... nan nan nan] [nan 964.4717 43.6671 ... nan nan nan] ... [nan 10524.1495 68.426 ... nan 32.4679 nan] [nan 9703.0137 44.2775 ... nan 31.8934 nan] [nan 10467.838 52.7504 ... nan 40.5009 nan]] [[nan 2746.5195 48.3787 ... nan 1.47 nan] [nan 2796.3674 45.6378 ... nan 1.47 nan] [nan 2874.8426 43.0037 ... nan 2.94 nan] ... [nan 5554.3619 79.8269 ... nan 0.4715 nan] [nan 7939.9378 111.4355 ... nan 0.6293 nan] [nan 10127.0484 101.3024 ... nan 0.7457 nan]]]
Units,CO2 gigagram/year
Magnitude,[[[3097.8213 9255.678 118.2797 ... 2368.1734 617.4 6.4546] [3063.8992 9331.0188 121.7607 ... 2326.1498 617.4 6.4546] [3116.2194 9620.3601 121.7607 ... 2390.2709 617.4 6.4546] ... [4732.3633 13832.9964 89.2428 ... 3159.8598 349.86 623.9758] [5717.29 16810.3052 101.4402 ... 4200.1834 679.7382 275.6073] [7962.9146 22644.1569 101.2144 ... 5780.8189 867.9956 511.3241]] [[95074.6375 233438.4999 1585.2062 ... 86108.4167 7199.4953 2277.8822] [96598.8864 237662.8733 1629.845 ... 86991.4047 8351.3734 2454.3328] [98584.3345 241883.7194 1654.9376 ... 88348.6382 8380.3971 2868.7402] ... [324796.6377 936234.3419 4101.287 ... 263095.5966 35342.1316 25091.0376] [319774.7461 881918.151 3806.1285 ... 250118.2627 30836.1898 31441.5307] [398749.5427 1039555.2658 4383.948 ... 303100.5699 37580.9168 46671.6549]] [[565.3851 1903.8378 16.7578 ... 378.0223 17.445999999

## comparing usage

### access: select the sector category

In [7]:
proposal1[proposal1.attrs['sector_dimension']]

<xarray.DataArray 'Sector (FAO terminology)' (Sector (FAO terminology): 12)>
array(['Agricultural Soils', 'Agriculture total', 'Burning - Crop residues',
       'Burning - Savanna', 'Crop Residues', 'Cultivation of Organic Soils',
       'Enteric Fermentation', 'Manure Management', 'Manure applied to Soils',
       'Manure left on Pasture', 'Rice Cultivation', 'Synthetic Fertilizers'],
      dtype=object)
Coordinates:
  * Sector (FAO terminology)  (Sector (FAO terminology)) object 'Agricultural...

In [8]:
proposal2['sector']

<xarray.DataArray 'sector' (sector: 12)>
array(['Agricultural Soils', 'Agriculture total', 'Burning - Crop residues',
       'Burning - Savanna', 'Crop Residues', 'Cultivation of Organic Soils',
       'Enteric Fermentation', 'Manure Management', 'Manure applied to Soils',
       'Manure left on Pasture', 'Rice Cultivation', 'Synthetic Fertilizers'],
      dtype=object)
Coordinates:
  * sector   (sector) object 'Agricultural Soils' ... 'Synthetic Fertilizers'

**For proposal1, accessors could be provided. With a dirty hack, I can show how that would look like**

In [9]:
# dirty hack, implementation would of course be somewhere in a library
@xr.register_dataset_accessor("primap")
class PrimapAccessor:
    def __init__(self, xarray_obj):
        self._obj = xarray_obj
    
    def loc(self, common_dim):
        return self._obj[self._obj.attrs[f'{common_dim}_dimension']]

In [10]:
# using the custom accessor, we can now access the sector dimension using the `sector` alias
proposal1.primap.loc('sector')

<xarray.DataArray 'Sector (FAO terminology)' (Sector (FAO terminology): 12)>
array(['Agricultural Soils', 'Agriculture total', 'Burning - Crop residues',
       'Burning - Savanna', 'Crop Residues', 'Cultivation of Organic Soils',
       'Enteric Fermentation', 'Manure Management', 'Manure applied to Soils',
       'Manure left on Pasture', 'Rice Cultivation', 'Synthetic Fertilizers'],
      dtype=object)
Coordinates:
  * Sector (FAO terminology)  (Sector (FAO terminology)) object 'Agricultural...